# 挂载网盘
/content/drive/MyDrive/code/ 用于存放代码

In [1]:
import os
from google.colab import drive
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')

Mounted at /content/drive


# 下载代码

In [2]:
# 如果 /content/drive/MyDrive/code/mcp-frp/ 不存在则下载 https://github.com/eatcosmos/mcp-frp
!mkdir -p /content/drive/MyDrive/code/
!rm -rf /content/drive/MyDrive/code/mcp-frp/
if not os.path.exists('/content/drive/MyDrive/code/mcp-frp/'):
  %cd /content/drive/MyDrive/code/
  !git clone https://github.com/eatcosmos/mcp-frp.git

/content/drive/MyDrive/code
Cloning into 'mcp-frp'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 49 (delta 16), reused 40 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (49/49), 9.30 MiB | 13.45 MiB/s, done.
Resolving deltas: 100% (16/16), done.


# 准备工作！
1. 准备一个新账号，淘宝充值Colab PRO
2. 把自己电脑常用的 id_rsa.pub 文件替换 /content/drive/MyDrive/code/mcp-frp/id_rsa.pub

In [3]:

!chmod +x /content/drive/MyDrive/code/mcp-frp/setupSSH.sh
#PRO版本在终端里面运行 /content/drive/MyDrive/code/mcp-frp/setupSSH.sh
!/content/drive/MyDrive/code/mcp-frp/setupSSH.sh
# !ssh -p 22 root@127.0.0.1

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQDP6FNjJvsc49car3uf7cVdHBBugKx7t6zqMdGubKq/bF3cPlDDbRu60PZOSRkgbC+HsCUSVfJv/QFajgqe7+nRYkeKEMEm40aBxJlzvPlB9ahrL/H55Wh6uSSjLQVs2k2xW7/k8YYyXWZPqoeTaerg0wxHE3YADpcbZJTSYf170hlbSbknVFlJJdAKM9Eq7yEATQoxoYzi479w7zC1Na4UI4puDyCcmH1FmYY7EfSCQdUd2vmXBHhrSn8KrJD9soA0pI4cN66EhqiO88Fqns3+yBXGayLDfp5abboO+JbBtLfFYxNtA32HPxwsU0lQkOP8tspCXNJdExFtCRoLZTHfPd7ITrEKAhTpNHwH9efjgrljjnVQb4is9d+347l2uhOya3JfdSYSi1LnaXstkHEvytnYs5oSiwybOprnVEfVfMqFIT/rr3GjxyTR9TEOX79t5ijWMKJGaeLQezEdrWx6scUMPidvojn+rH2X/NZ6vU4CLU9F93aP5aRRVNl91EE= ubuntu@LAPTOP-FSPULTNV


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)



Creating config file /etc/ssh/sshd_config with new version
Creating SSH2 RSA key; this may take some time ...
3072 SHA256:h2tWvFYqn7reXAaD6udCnsHoa6kd+WfBUpVNzVTbHeU root@a2a7a4b9603d (RSA)
Creating SSH2 ECDSA key; this may take s

In [4]:
from google.colab import userdata

%cd /content/drive/MyDrive/code/mcp-frp/
!cp ./frp_0.63.0_linux_amd64/frpc-template.toml ./frp_0.63.0_linux_amd64/frpc.toml
# 替换 ./frp_0.63.0_linux_amd64/frpc.toml 里面 serverAddr = "127.0.0.1" 为 serverAddr = userdata.get('FRPS_SERVER_IP')
# 获取FRPS服务器IP
frps_ip = userdata.get('FRPS_SERVER_IP')

# 定义文件路径
file_path = './frp_0.63.0_linux_amd64/frpc.toml'

# 读取配置文件内容
with open(file_path, 'r') as file:
    content = file.read()

# 替换服务器地址
new_content = content.replace(
    'serverAddr = "127.0.0.1"',
    f'serverAddr = "{frps_ip}"'
)

# 写回配置文件
with open(file_path, 'w') as file:
    file.write(new_content)

print("配置文件更新完成！")

/content/drive/MyDrive/code/mcp-frp
配置文件更新完成！


In [5]:
if not os.path.exists('/content/drive/MyDrive/code/.cursor/'):
  !mkdir -p /content/drive/MyDrive/code/.cursor
if not os.path.exists('/content/drive/MyDrive/code/.cursor-server/'):
  !mkdir -p /content/drive/MyDrive/code/.cursor-server
!rm -rf /root/.cursor
!ln -sf /content/drive/MyDrive/code/.cursor /root/
!rm -rf /root/.cursor-server
!ln -sf /content/drive/MyDrive/code/.cursor-server /root/
#
!ls -la /root/.cursor-server

lrwxrwxrwx 1 root root 42 Jul  3 08:15 /root/.cursor-server -> /content/drive/MyDrive/code/.cursor-server


In [6]:
%cd /content/drive/MyDrive/code/mcp-frp/frp_0.63.0_linux_amd64/
!chmod +x ./frpc
# !cat ./frpc.toml
# !./frpc -c ./frpc.toml
!nohup ./frpc -c ./frpc.toml >/dev/null 2>&1 &

/content/drive/MyDrive/code/mcp-frp/frp_0.63.0_linux_amd64
^C


如果无法访问可以在自己的笔记本上运行下
ssh -p 7502 root@FRPS_SERVER_IP